In [1]:
%%time
import os, sys
import numpy as np
import pandas as pd
import seaborn as sns

#for plot with ATLAS styple
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

import mplhep as hep
#hep.style.use(hep.style.ROOT) # For now ROOT defaults to CMS
# Or choose one of the experiment styles
hep.style.use(hep.style.ATLAS)
plt.style.use(hep.style.ATLAS)
#be able to read root files.
#and convert to numpy or pandas
import uproot
import gc

from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2


import numpy as np
import awkward as ak
import uproot4 

h5_files = "/Users/abletimin/cernbox/Phase2_MET_Trigger_project/re21.9/local_run/subTask-0/myfile_tree.root"
tree_name = "ntuple"

METS = ["MET_Calo_pt", "MET_Calo_px", "MET_Calo_py",
                "met_truth_pt", "met_truth_px", "met_truth_py",
                "pufitCalo422_pt","pufitCalo422_px", "pufitCalo422_py"]
cluster_name = "Calo422TopoClusters"
cl_vars = ['et', 'eta', 'phi', 'n_cell', 'em_prob',
                'had_weight', 'significance', 'eng_frac_max', 'eng_frac_em',
                'cell_sig']
expressions= [f'{cluster_name}_{var}' for var in cl_vars]
expressions= expressions + METS # include met branches



CPU times: user 1.52 s, sys: 240 ms, total: 1.76 s
Wall time: 3.46 s


In [2]:
events_list = list()
_labels = {}
for met in METS:
        _labels[met] = []
        
for batch in uproot.iterate(h5_files+":"+tree_name, expressions=expressions,
                                step_size='1 GB', library="np"):
    for i in range(len(batch[f"{cluster_name}_{cl_vars[0]}"])):
            event_array = list()
            for var in cl_vars:
                event_array += [batch[f"{cluster_name}_{var}"][i]]
            events_list += [np.stack(event_array, axis=0).flatten()]
            
    for met in METS:
        _labels[met] += [batch[met]] 
                         
for met in METS:       
    print(np.concatenate(_labels[met], axis=0).shape)
    

(2000,)
(2000,)
(2000,)
(2000,)
(2000,)
(2000,)
(2000,)
(2000,)
(2000,)


In [3]:
events = np.array(events_list, dtype=object)

In [4]:
events_list[0].shape

(6510,)

In [5]:
import h5py
h5f = h5py.File("test.h5", 'w')


In [6]:
hdt = h5py.special_dtype(vlen=np.float32)
h5f.create_dataset('tcl3', data=events, dtype=hdt)
#for met in METS[:1]:
data=np.concatenate(_labels[METS[1]], axis=0)
h5f.create_dataset('xx', data=data)

<HDF5 dataset "xx": shape (2000,), type "<f4">

In [12]:
h5f['tcl3'][0].reshape(10,-1)[1:2,0]

array([0.5175891], dtype=float32)

In [58]:
h5f['tcl3'][1].reshape(1, 10,-1)

array([[[37.93805   , 26.717224  , 22.304832  , ..., -1.0379906 ,
         -1.2876918 , -1.5689633 ],
        [ 4.2010865 ,  2.8439364 , -1.7859802 , ...,  0.56581485,
         -0.4512145 ,  0.91662157],
        [ 0.39478514, -2.349417  ,  1.6677331 , ..., -0.23658468,
         -2.8978221 ,  1.1507704 ],
        ...,
        [ 0.11057816,  0.14452043,  0.19384487, ...,  0.        ,
          1.        ,  0.        ],
        [ 0.3476431 ,  1.        ,  0.9723732 , ...,  0.        ,
          1.        ,  0.        ],
        [ 8.981715  ,  6.7440047 , 59.43528   , ...,  0.        ,
         -4.4414644 ,  0.        ]]], dtype=float32)

In [63]:
np.transpose(h5f['tcl3'][1].reshape(1, 10,-1), (0,2,1)).shape

(1, 841, 10)

# Model

In [73]:
from tensorflow.keras.layers import Conv2D, TimeDistributed,Dropout,Input, Dense,\
    BatchNormalization, GRU, Layer, Flatten, Conv1D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam

import  tensorflow as tf


In [142]:
def data_converter(input_arrays=h5f['tcl3'][:]):
    for x in input_arrays:
        yield np.transpose(x.reshape(1, 10,-1), (0, 2,1))

dataset = tf.data.Dataset.from_generator(data_converter,
                                         output_types=np.float32)

batched_dataset = dataset.batch(4)
#dataset_reshaped = dataset.map(lambda x: x.numpy().reshape(10,-1))

In [119]:
for d in dataset:
    d

In [149]:
inputs = Input(shape=(None, 10))
conv1d = Conv1D(100, 1, activation='relu', input_shape=(None,10))(inputs)
SUM = tf.keras.layers.Lambda(lambda s: tf.reduce_sum(s, axis=-2))(conv1d)
outputs = tf.keras.layers.ReLU()(SUM)
model = Model(inputs=inputs,outputs=outputs)

#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [144]:
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, None, 10)]        0         
                                                                 
 conv1d_20 (Conv1D)          (None, None, 100)         1100      
                                                                 
 lambda_8 (Lambda)           (None, 100)               0         
                                                                 
Total params: 1,100
Trainable params: 1,100
Non-trainable params: 0
_________________________________________________________________


In [145]:
model.predict(np.transpose(h5f['tcl3'][1].reshape(1, 10,-1), (0,2,1))).shape

1/1 [==============================] - 0s 30ms/step


(1, 100)

In [150]:
results = model.predict(dataset)

2000/2000 [==============================] - 1s 697us/step


In [151]:
results[0]

array([1.33604250e+05, 0.00000000e+00, 7.78721328e+04, 0.00000000e+00,
       0.00000000e+00, 1.58896719e+05, 1.97182988e+04, 1.50145898e+04,
       9.22774141e+04, 2.19252734e+04, 0.00000000e+00, 9.96846641e+04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.15331055e+03,
       0.00000000e+00, 1.25086258e+05, 1.05593516e+05, 0.00000000e+00,
       0.00000000e+00, 9.28271057e+02, 0.00000000e+00, 0.00000000e+00,
       1.25790078e+05, 0.00000000e+00, 1.40069250e+05, 0.00000000e+00,
       1.03257617e+05, 1.07703391e+05, 0.00000000e+00, 7.98318047e+04,
       0.00000000e+00, 4.00211487e+01, 0.00000000e+00, 1.38584703e+05,
       2.00628688e+05, 2.08360406e+05, 1.30154602e+05, 3.45033516e+04,
       0.00000000e+00, 1.54015453e+05, 1.44014541e+04, 5.98734531e+04,
       0.00000000e+00, 9.14488047e+04, 0.00000000e+00, 1.61217047e+05,
       6.29263945e+04, 0.00000000e+00, 0.00000000e+00, 1.73684609e+05,
       2.60877075e+01, 0.00000000e+00, 4.80408750e+04, 1.07496201e+04,
      